# Getting started with EHRData

<img src="../_static/tutorial_images/logo.png" width="600" height="500" alt="Logo">

`EHRData` is building upon and extending [AnnData](https://anndata.readthedocs.io/en/stable/), a Python package for handling annotated data that is commonly used for sequencing data, to further support time series data by representing data of $n$ observations of $d$ variables of $t$ repeats.
It is the datastructure that [ehrapy](https://ehrapy.readthedocs.io/en/stable/) operates on.

In clinical studies, each enrolled subject corresponds to an observation, each registered clinical parameter corresponds to a variable, and each visit corresponds to a repeat.
Furthermore, we might have metadata for each of these axis.
For example, for each subject, we might have additional static metadata, such as birthdata, or sex. For each registered clinical parameter, we might have metadata such as a concept identifier, a descriptive name, or the unit it was measured in. For the repeated measurements, we might have a descriptive name per measurement, or the number of weeks after study entry.


```{note}

If you are new to Anndata, we recommend starting for example with the brief tutorial [Getting started with AnnData](https://anndata.readthedocs.io/en/stable/tutorials/notebooks/getting-started.html).

```

## Initializing EHRData

In [3]:
import numpy as np
import pandas as pd
import ehrdata as ed

Let's start by building a basic EHRData object with some measurements, e.g. blood pressure of two individuals.

In [4]:
patients = pd.DataFrame(
    {"patient_id": ["P001", "P002"], "birthdate": ["1980-01-01", "1975-05-15"], "gender": ["M", "F"]}
).set_index("patient_id")

clinical_parameters = pd.DataFrame(
    {
        "parameter_id": ["BP_Systolic", "BP_Diastolic"],
        "name": ["Systolic Blood Pressure", "Diastolic Blood Pressure"],
        "unit": ["mmHg", "mmHg"],
    }
).set_index("parameter_id")

measurements = np.array(
    [[120, 121], [81, 81]],
)

edata = ed.EHRData(
    X=measurements,
)
edata

EHRData object with n_obs × n_vars = 2 × 2
    shape of .X: (2, 2)

EHRData supports flat 2D matrix field `edata.X`, suitable for regular tabular variable representation.

In [5]:
edata.X

array([[120, 121],
       [ 81,  81]])

When we have measurements along a time course, we want to represent an axis of time (e.g. clinical visits, calendar time, ...) and repeats of measurements.
In the example above, the measurements could e.g. be repeated three times.

In [6]:
visit_dates = pd.DataFrame({"visit_number": ["1", "2", "3"], "visit_id": ["V001", "V002", "V003"]}).set_index(
    "visit_number"
)

repeated_measurements = np.array(
    [
        [
            [120, np.nan, 121],
            [81, np.nan, 81],
        ],
        [
            [130, 135, 125],
            [84, 81, 80],
        ],
    ]
)

edata = ed.EHRData(
    R=repeated_measurements,
)
edata

EHRData object with n_obs × n_vars × n_t = 2 × 2 × 3
    shape of .X: (2, 2)
    shape of .R: (2, 2, 3)

EHRData provides a 3D array field `edata.R`, suitable for this kind of data.

In [7]:
edata.R

array([[[120.,  nan, 121.],
        [ 81.,  nan,  81.]],

       [[130., 135., 125.],
        [ 84.,  81.,  80.]]])

Now, we group the data together with its metadata, using the `obs`, `var`, and `tem` fields of EHRData.

- the `obs` field stores static person-level metadata
- the `var` field stores variable-level metadata
- the `tem` field stores time axis-level metadata

In [8]:
edata = ed.EHRData(
    R=repeated_measurements,
    obs=patients,
    var=clinical_parameters,
    tem=visit_dates,
)
edata

EHRData object with n_obs × n_vars × n_t = 2 × 2 × 3
    obs: 'birthdate', 'gender'
    var: 'name', 'unit'
    tem: '1', '2', '3'
    shape of .X: (2, 2)
    shape of .R: (2, 2, 3)

### Subsetting EHRData
#### Subsetting with indices

The index values can be used to subset the EHRData, which provides a view of the EHRData object.
We can imagine this to be useful to subset the AnnData to particular patients, variables, or time intervals of interest.
The rules for subsetting EHRData are quite similar to that of a Pandas DataFrame.
You can use values in the `obs/var_names`, boolean masks, or cell index integers.

In [9]:
edata.var_names.isin(["P001", "BP_Systolic"])

array([ True, False])

In [10]:
edata[:, edata.var_names.isin(["P001", "BP_Systolic"])]

View of EHRData object with n_obs × n_vars × n_t = 2 × 1 × 3
    obs: 'birthdate', 'gender'
    var: 'name', 'unit'
    tem: '1', '2', '3'
    shape of .X: (2, 1)
    shape of .R: (2, 1, 3)

#### Subsetting using metadata

We can also subset the EHRData using the metadata:

In [11]:
edata[edata.obs["gender"] == "F"]

View of EHRData object with n_obs × n_vars × n_t = 1 × 2 × 3
    obs: 'birthdate', 'gender'
    var: 'name', 'unit'
    tem: '1', '2', '3'
    shape of .X: (1, 2)
    shape of .R: (1, 2, 3)

## Observation/variable-level matrices

We might also have metadata at either level that has many dimensions to it, such as a UMAP embedding of the data.
For this type of metadata, EHRData has the `.obsm/.varm` attributes.
We use keys to identify the different matrices we insert.
The restrictions of `.obsm/.varm` are, that the length of `.obsm` matrices must be equal to the number of observations as `.n_obs` and `.varm` matrices must be equal in length to `.n_vars`.
They can each independently have a different number of dimensions.

Let's start with a randomly generated matrix that we can interpret as a UMAP embedding of the data we would like to store, as well as some random variable-level metadata:

In [28]:
edata.obsm["X_umap"] = np.random.normal(0, 1, size=(edata.n_obs, 2))
edata.varm["variable_stuff"] = np.random.normal(0, 1, size=(edata.n_vars, 5))
edata.obsm

AxisArrays with keys: X_umap

A few more notes about `.obsm/.varm`

1. The "array-like" metadata can originate from a Pandas DataFrame, scipy sparse matrix, or numpy dense array.
2. When using scanpy, their values (columns) are not easily plotted, where instead items from `.obs` are easily plotted on, e.g., UMAP plots.

## Unstructured metadata

EHRData has `.uns`, which allows for any unstructured metadata. This can be anything, like a list or a dictionary with some general information that was useful in the analysis of our data.

In [29]:
edata.uns["random"] = [1, 2, 3]
edata.uns

OrderedDict([('random', [1, 2, 3])])

## Layers

Finally, we may have different forms of our original core data, perhaps one that is normalized and one that is not. These can be stored in different layers in EHRData. For example, let's log transform the original data and store it in a layer:

In [30]:
edata.layers["log_transformed"] = np.log1p(edata.R)
edata

EHRData object with n_obs × n_vars × n_t = 2 × 2 × 3
    obs: 'birthdate', 'gender'
    var: 'name', 'unit'
    tem: '1', '2', '3'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'variable_stuff'
    layers: 'log_transformed'
    shape of .X: (2, 2)
    shape of .R: (2, 2, 3)

## Writing the results to disk

`EHRData` comes with a persistent HDF5-based file format: `h5ad`. If string columns with small number of categories aren't yet categoricals, `EHRData` will auto-transform to categoricals.

In [31]:
edata.write("my_results.h5ad", compression="gzip")

In [21]:
df_long = pd.DataFrame(
    {
        "observation_id": ["0", "0", "1", "1"],
        "variable": [
            "systolic_blood_pressure",
            "systolic_blood_pressure",
            "systolic_blood_pressure",
            "systolic_blood_pressure",
        ],
        "time": ["t_0", "t_1", "t_0", "t_2"],
        "value": [120, 125, 130, 135],
    }
)
df_long

,observation_id,variable,time,value
0,0,systolic_blood_pressure,t_0,120
1,0,systolic_blood_pressure,t_1,125
2,1,systolic_blood_pressure,t_0,130
3,1,systolic_blood_pressure,t_2,135


This dataframe format can easily be ingested into the `EHRData` format...

In [31]:
edata = ed.io.from_pandas(df_long, format="long")
edata

EHRData object with n_obs × n_vars × n_t = 2 × 1 × 3
    shape of .X: (2, 1)
    shape of .R: (2, 1, 3)

... and from the `EHRData` format can be transformed back to a pandas DataFrame.

In [34]:
df_long_from_edata = ed.io.to_pandas(edata, layer="R", format="long")
df_long_from_edata

,observation_id,variable,time,value
0,0,systolic_blood_pressure,t_0,120.0
1,0,systolic_blood_pressure,t_1,125.0
2,0,systolic_blood_pressure,t_2,NaN
3,1,systolic_blood_pressure,t_0,130.0
4,1,systolic_blood_pressure,t_1,NaN
5,1,systolic_blood_pressure,t_2,135.0


### The `wide` format

In the `wide` format, the data is stored in a dataframe with rows indicating the person, and columns indicating the variable as well as the time of measurement.

In [35]:
df_wide = pd.DataFrame(
    {
        "patient_id": ["0", "1"],
        "sex": ["F", "M"],
        "systolic_bp_t_0": [120, 130],
        "systolic_bp_t_2": [125, np.nan],  # the suffix strings are sorted lexicographically
        "systolic_bp_t_1": [np.nan, 135],
    }
)
df_wide

,patient_id,sex,systolic_bp_t_0,systolic_bp_t_2,systolic_bp_t_1
0,0,F,120,125.0,NaN
1,1,M,130,NaN,135.0


This dataframe format can easily be ingested into the `EHRData` format...

In [41]:
edata_wide = ed.io.from_pandas(df_wide, format="wide", columns_obs_only=["patient_id", "sex"])
edata_wide

EHRData object with n_obs × n_vars × n_t = 2 × 1 × 3
    obs: 'patient_id', 'sex'
    shape of .X: (2, 1)
    shape of .R: (2, 1, 3)

... and from the `EHRData` format can be transformed back to a pandas DataFrame.

In [45]:
df_from_edata = ed.io.to_pandas(edata_wide, layer="R", format="wide", obs_cols=["patient_id", "sex"])
df_from_edata

,systolic_bp_t_0,systolic_bp_t_1,systolic_bp_t_2,patient_id,sex
0,120.0,NaN,125.0,0,F
1,130.0,135.0,NaN,1,M


## Wrapping up the introduction

EHRData is straightforward to use and faciliatates more reproducible analyses with it's key-based storage.

We refer to the [AnnData tutorials](https://anndata.readthedocs.io/en/stable/tutorials/notebooks/getting-started.html) to better understand "views", on-disk backing, and other details.

```{Note}

Similar to numpy arrays, EHRData objects can either hold actual data or reference another `EHRData` object. In the later case, they are referred to as "view".

Subsetting EHRData objects always returns views, which has two advantages:

- no new memory is allocated
- it is possible to modify the underlying EHRData object

You can get an actual EHRData object from a view by calling `.copy()` on the view. Usually, this is not necessary, as any modification of elements of a view (calling `.[]` on an attribute of the view) internally calls `.copy()` and makes the view an EHRData object that holds actual data. See the example below.
    
```

Get access to the first 5 rows for two variables.

```{Note}

Indexing into AnnData will assume that integer arguments to `[]` behave like `.iloc` in pandas, whereas string arguments behave like `.loc`. `AnnData` always assumes string indices.
    
```

In [32]:
edata[["P001"], ["BP_Systolic"]]

View of EHRData object with n_obs × n_vars × n_t = 1 × 1 × 3
    obs: 'birthdate', 'gender'
    var: 'name', 'unit'
    tem: '1', '2', '3'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'variable_stuff'
    layers: 'log_transformed'
    shape of .X: (1, 1)
    shape of .R: (1, 1, 3)

This is a view! If we want an `EHRData` that holds the data in memory, we have to call `.copy()`

In [33]:
edata_subset = edata[["P001"], ["BP_Systolic"]].copy()

If you try to write to parts of a view of an AnnData, the content will be auto-copied and a data-storing object will be generated.

In [34]:
edata_subset = edata[["P001"], ["BP_Systolic"]]
edata_subset

View of EHRData object with n_obs × n_vars × n_t = 1 × 1 × 3
    obs: 'birthdate', 'gender'
    var: 'name', 'unit'
    tem: '1', '2', '3'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'variable_stuff'
    layers: 'log_transformed'
    shape of .X: (1, 1)
    shape of .R: (1, 1, 3)

In [35]:
edata_subset.obs["foo"] = "bar"

/tmp/ipykernel_190481/1547146112.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  edata_subset.obs["foo"] = "bar"


Now `edata_subset` stores the actual data and is no longer just a reference to `edata`.

In [36]:
edata_subset

EHRData object with n_obs × n_vars × n_t = 1 × 1 × 3
    obs: 'birthdate', 'gender', 'foo'
    var: 'name', 'unit'
    tem: '1', '2', '3'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'variable_stuff'
    layers: 'log_transformed'
    shape of .X: (1, 1)
    shape of .R: (1, 1, 3)